<a href="https://colab.research.google.com/github/kwanda2426/projects/blob/main/Sebokeng_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries**

In [10]:
!pip install pandas openpyxl


In [11]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
#
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from statsmodels.graphics.correlation import plot_corr
from scipy.stats import skew
from scipy.stats import kurtosis
import statistics
import missingno as msn

#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

#making sure that we can see all rows and cols
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


# **Importing Data**

In [18]:
# Reading data
url = "https://raw.githubusercontent.com/kwanda2426/projects/main/sebokeng_data.xlsx"
df = pd.read_excel(url, engine='openpyxl')


In [20]:
# Showing first 5 lines of data
df.head()


,sebSO2,sebNO2,sebO3,sebPM25,sebPM10
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [23]:
# Rows and Columns of data
print('Data has {} rows and {} Columns'.format(df.shape[0],df.shape[1]))
print('')
print('Data has the following columns:')
for i in df.columns:
  print(i)

Data has 3345 rows and 5 Columns

Data has the following columns:
sebSO2
sebNO2
sebO3
sebPM25
sebPM10


### Data Type of Columns

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3345 entries, 0 to 3344
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sebSO2   1925 non-null   float64
 1   sebNO2   2055 non-null   float64
 2   sebO3    2371 non-null   float64
 3   sebPM25  1853 non-null   float64
 4   sebPM10  1827 non-null   float64
dtypes: float64(5)
memory usage: 130.8 KB


We can see that all our columns have the same data type which is numeric. We can see that non-null rows for each column is not the same as total rows, this means that our data has some missing entries.

In [34]:
for i in df.columns:
  print('{} has {} rows with missing values',format(i,str(df[[i]].isna().sum())))


ValueError: Invalid format specifier 'sebSO2    1420
dtype: int64' for object of type 'str'

# New Section